In [3]:
import boto3 
from pprint import pprint
import csv
import logging
from botocore.exceptions import ClientError
import json

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

dynamodb = boto3.session.Session(profile_name="shared").resource(region_name="us-east-1",service_name="dynamodb")
client = boto3.session.Session(profile_name="shared").client(region_name="us-east-1",service_name="dynamodb")

In [2]:
# creating a dynamodb table

def create_dynamodb(TableName: str):
    try:
        response = client.create_table(
            TableName= TableName,
            KeySchema=[
                {
                    'AttributeName': 'ID',
                    'KeyType': 'HASH'
                },
        
            ],
            AttributeDefinitions=[
                {
                    'AttributeName': 'ID',
                    'AttributeType': 'N'
                },
          
            ],
            ProvisionedThroughput= {
                'ReadCapacityUnits': 1,
                'WriteCapacityUnits': 1
            },

            Tags=[
                {
                    'Key': 'Name',
                    'Value': 'kojitechs-table'
                },
            ],
        )
        logger.info("Table Created Successfully")
        print("Table Created Successfully")
    except Exception as err:
        if err.response['Error']['Code']== 'ResourceInUseException':
            logger.error("Table already exits")
        else:
            logger.error(err.response['Error'])
    except ClientError as err:
        logger.error(err)
    


In [5]:
create_dynamodb(TableName='kojitechs-json')

Table already exits


## Read that json file and dump the content to dynamo


In [19]:
path = '/Users/kojibello/Downloads/GitLAP/aws-cdk-learning/Mega-python-course/supermarkets/supermarkets.json'


def batch_write(tableName: str):
    if tableName is None:
        tableName = 'kojitechs-json'
    table = dynamodb.Table(tableName)
    
    # opening json file
    try:
        with open(path, 'r') as file:
            data = file.read()
        json_object = json.loads(data)
        # open dynamodb table and dump and json obj
        with table.batch_writer() as batch:
            for each_row in json_object:
                batch.put_item(Item =each_row )
        print("done")
    except Exception as err:
        logger.error(err)
            
batch_write(None)
    

done


In [ ]:


import boto3
from botocore.exceptions import ClientError
import botocore
import json
import os
import logging

logger = logging.getLogger()
logger.setLevel(logging.DEBUG)


s3 = boto3.client('s3')
dynamodb = boto3.resource(region_name="us-east-1",service_name="dynamodb")
client = boto3.client(region_name="us-east-1",service_name="dynamodb")


def lambda_handler(event, context):
    bucket = event['Records'][0]['s3']['bucket']['name']
    bucket_object = event['Records'][0]['s3']['object']['key']
    response = s3.get_object(Bucket=bucket, Key=bucket_object)
    
    # converting from byte to string
    data_string = response['Body'].read().decode('utf-8')
    logger.info(data_string)
    logger.info(type(data_string))
    
    # converting from string to dictionary
    data_dict = json.loads(data_string)
    logger.info(data_dict)
    logger.info(type(data_dict))
    
    tableName = os.environ['table']
    # creating dynamodb table
    create_dynamodb(TableName= tableName)
    
    # load json data to dynamodb
    table = dynamodb.Table(tableName)
    # opening json file
    try:
        # open dynamodb table and dump and json obj
        with table.batch_writer() as batch:
            for each_row in data_dict:
                batch.put_item(Item =each_row )
        print("done")
    except Exception as err:
        logger.error(err)
        
    

# creating a dynamodb table
def create_dynamodb(TableName: str):
    try:
        response = client.create_table(
            TableName= TableName,
            KeySchema=[
                {
                    'AttributeName': 'ID',
                    'KeyType': 'HASH'
                },
        
            ],
            AttributeDefinitions=[
                {
                    'AttributeName': 'ID',
                    'AttributeType': 'N'
                },
          
            ],
            ProvisionedThroughput= {
                'ReadCapacityUnits': 1,
                'WriteCapacityUnits': 1
            },

            Tags=[
                {
                    'Key': 'Name',
                    'Value': 'kojitechs-table'
                },
            ],
        )
        logger.info("Table Created Successfully {}".format(response))
        print("Table Created Successfully")
    except Exception as err:
        if err.response['Error']['Code']== 'ResourceInUseException':
            logger.error("Table already exits")
        else:
            logger.error(err.response['Error'])
    except ClientError as err:
        logger.error(err)
        